In [9]:
import pandas as pd
from pymatgen.io.cif import CifParser
from pyxtal import pyxtal
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


In [2]:
csv_path = "/home/holywater2/crystal_gen/mattergen/datasets/alex_mp_20"

In [3]:
df = pd.read_csv(csv_path + "/val.csv")

In [4]:
df.cif[0]

"# generated using pymatgen\ndata_Ho3Pd4\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   5.74995063\n_cell_length_b   7.85273876\n_cell_length_c   7.85273876\n_cell_angle_alpha   114.24515457\n_cell_angle_beta   104.12712300\n_cell_angle_gamma   104.12712300\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Ho3Pd4\n_chemical_formula_sum   'Ho6 Pd8'\n_cell_volume   288.77694651\n_cell_formula_units_Z   2\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Ho  Ho0  1  0.27521594  0.87038120  0.25129853  1\n  Ho  Ho1  1  0.40483475  0.38091733  0.12961880  1\n  Ho  Ho2  1  0.02391741  0.74870147  0.61908267  1\n  Ho  Ho3  1  0.72478406  0.12961880  0.74870147  1\n  Ho  Ho4  1  0.59516525  0.61908267  0.87038120  1\n  Ho  Ho5  1  0.97608259  0.25129853  0.38091733

In [5]:
sturctures = CifParser.from_str(df.cif[0]).get_structures()[0]

/tmp/ipykernel_1910131/2486049076.py:1: FutureWarning: get_structures is deprecated; use parse_structures in pymatgen.io.cif instead.
The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  sturctures = CifParser.from_str(df.cif[0]).get_structures()[0]


In [ ]:
def get_symmetry_info(crystal, tol=0.01):
    spga = SpacegroupAnalyzer(crystal, symprec=tol)
    crystal = spga.get_refined_structure()
    c = pyxtal()
    try:
        c.from_seed(crystal, tol=0.01)
    except:
        c.from_seed(crystal, tol=0.0001)
    space_group = c.group.number
    species = []
    anchors = []
    matrices = []
    coords = []
    for site in c.atom_sites:
        specie = site.specie
        anchor = len(matrices)
        coord = site.position
        for syms in site.wp:
            species.append(specie)
            matrices.append(syms.affine_matrix)
            coords.append(syms.operate(coord))
            anchors.append(anchor)
    anchors = np.array(anchors)
    matrices = np.array(matrices)
    coords = np.array(coords) % 1.
    sym_info = {
        'anchors':anchors,
        'wyckoff_ops':matrices,
        'spacegroup':space_group
    }
    crystal = Structure(
        lattice=Lattice.from_parameters(*np.array(c.lattice.get_para(degree=True))),
        species=species,
        coords=coords,
        coords_are_cartesian=False,
    )
    return crystal, sym_info


In [10]:
c = pyxtal()

In [11]:
c.from_seed(sturctures, tol=0.01)

In [40]:
sturctures

Structure Summary
Lattice
    abc : 5.74995063 7.85273876 7.85273876
 angles : 114.24515456999998 104.127123 104.127123
 volume : 288.77694645306684
      A : 5.576052474024184 0.0 -1.4034140708879075
      B : -2.7880262338170163 6.5949957177590015 -3.2246623443540385
      C : 0.0 0.0 7.85273876
    pbc : True True True
PeriodicSite: Ho0 (Ho) (-0.892, 5.74, -1.22) [0.2752, 0.8704, 0.2513]
PeriodicSite: Ho1 (Ho) (1.195, 2.512, -0.7786) [0.4048, 0.3809, 0.1296]
PeriodicSite: Ho2 (Ho) (-1.954, 4.938, 2.414) [0.02392, 0.7487, 0.6191]
PeriodicSite: Ho3 (Ho) (3.68, 0.8548, 4.444) [0.7248, 0.1296, 0.7487]
PeriodicSite: Ho4 (Ho) (1.593, 4.083, 4.003) [0.5952, 0.6191, 0.8704]
PeriodicSite: Ho5 (Ho) (4.742, 1.657, 0.811) [0.9761, 0.2513, 0.3809]
PeriodicSite: Pd6 (Pd) (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Pd7 (Pd) (2.788, 0.0, -0.7017) [0.5, 0.0, 0.0]
PeriodicSite: Pd8 (Pd) (2.17, 2.11, 2.008) [0.5492, 0.3199, 0.4852]
PeriodicSite: Pd9 (Pd) (0.8173, 1.09, 4.486) [0.2292, 0.1652, 0.6801

In [36]:
c


------Crystal from Seed------
Dimension: 3
Composition: Ho18Pd24
Group: R -3:H (148)
 13.1900,  13.1900,   5.7500,  90.0000,  90.0000, 120.0000, trigonal
Wyckoff sites:
	Ho @ [ 0.3739  0.8774  0.9013], WP [18f] Site [1]
	Pd @ [ 0.0000  0.0000  0.0000], WP [3a] Site [-3.]
	Pd @ [ 0.0000  0.0000  0.5000], WP [3b] Site [-3.]
	Pd @ [ 0.2684  0.2168  0.2808], WP [18f] Site [1]

In [ ]:
c.to_file

In [35]:
rs = SpacegroupAnalyzer(sturctures).get_refined_structure()

In [39]:
rs

Structure Summary
Lattice
    abc : 13.189991434672992 13.189991434672992 5.74995063
 angles : 90.0 90.0 120.00000000000001
 volume : 866.3308393221997
      A : 13.189991434672992 0.0 0.0
      B : -6.594995717336496 11.422867658125964 0.0
      C : 0.0 0.0 5.74995063
    pbc : True True True
PeriodicSite: Ho (-0.8548, 10.02, 5.183) [0.3739, 0.8774, 0.9013]
PeriodicSite: Ho (-4.083, 10.96, 1.349) [0.1702, 0.9594, 0.2347]
PeriodicSite: Ho (4.938, 1.864, 3.266) [0.4559, 0.1632, 0.568]
PeriodicSite: Ho (0.8548, 5.208, 2.484) [0.2928, 0.4559, 0.432]
PeriodicSite: Ho (4.083, 4.271, 0.5674) [0.4965, 0.3739, 0.09868]
PeriodicSite: Ho (1.657, 1.944, 4.401) [0.2107, 0.1702, 0.7653]
PeriodicSite: Ho (-0.8548, 2.407, 1.349) [0.04056, 0.2107, 0.2347]
PeriodicSite: Ho (9.107, 3.344, 3.266) [0.8368, 0.2928, 0.568]
PeriodicSite: Ho (-1.657, 5.671, 5.183) [0.1226, 0.4965, 0.9013]
PeriodicSite: Ho (7.45, 9.016, 4.401) [0.9594, 0.7893, 0.7653]
PeriodicSite: Ho (-2.512, 8.079, 2.484) [0.1632, 0.7072, 0.

In [30]:
d = pyxtal()

In [37]:
d.from_seed(rs, tol=0.01)

In [38]:
d


------Crystal from Seed------
Dimension: 3
Composition: Ho18Pd24
Group: R -3:H (148)
 13.1900,  13.1900,   5.7500,  90.0000,  90.0000, 120.0000, trigonal
Wyckoff sites:
	Ho @ [ 0.0406  0.2107  0.2347], WP [18f] Site [1]
	Pd @ [ 0.0000  0.0000  0.0000], WP [3a] Site [-3.]
	Pd @ [ 0.0000  0.0000  0.5000], WP [3b] Site [-3.]
	Pd @ [ 0.2684  0.2168  0.2808], WP [18f] Site [1]

In [13]:
species = []
anchors = []
matrices = []
coords = []
for site in c.atom_sites:
    print(site.specie, site.wp)
    specie = site.specie
    anchor = len(matrices)
    coord = site.position
    
    

Ho Wyckoff position 18f in space group 148 with site symmetry 1
x, y, z
-y, x-y, z
-x+y, -x, z
-x, -y, -z
y, -x+y, -z
x-y, x, -z
x+2/3, y+1/3, z+1/3
-y+2/3, x-y+1/3, z+1/3
-x+y+2/3, -x+1/3, z+1/3
-x+2/3, -y+1/3, -z+1/3
y+2/3, -x+y+1/3, -z+1/3
x-y+2/3, x+1/3, -z+1/3
x+1/3, y+2/3, z+2/3
-y+1/3, x-y+2/3, z+2/3
-x+y+1/3, -x+2/3, z+2/3
-x+1/3, -y+2/3, -z+2/3
y+1/3, -x+y+2/3, -z+2/3
x-y+1/3, x+2/3, -z+2/3
Pd Wyckoff position 3a in space group 148 with site symmetry -3.
0, 0, 0
2/3, 1/3, 1/3
1/3, 2/3, 2/3
Pd Wyckoff position 3b in space group 148 with site symmetry -3.
0, 0, 1/2
2/3, 1/3, 5/6
1/3, 2/3, 7/6
Pd Wyckoff position 18f in space group 148 with site symmetry 1
x, y, z
-y, x-y, z
-x+y, -x, z
-x, -y, -z
y, -x+y, -z
x-y, x, -z
x+2/3, y+1/3, z+1/3
-y+2/3, x-y+1/3, z+1/3
-x+y+2/3, -x+1/3, z+1/3
-x+2/3, -y+1/3, -z+1/3
y+2/3, -x+y+1/3, -z+1/3
x-y+2/3, x+1/3, -z+1/3
x+1/3, y+2/3, z+2/3
-y+1/3, x-y+2/3, z+2/3
-x+y+1/3, -x+2/3, z+2/3
-x+1/3, -y+2/3, -z+2/3
y+1/3, -x+y+2/3, -z+2/3
x-y+1/3, x+2/

In [28]:
sym = SpacegroupAnalyzer(sturctures).get_symmetrized_structure()

In [31]:
sym

SymmetrizedStructure
Full Formula (Ho6 Pd8)
Reduced Formula: Ho3Pd4
Spacegroup: R-3 (148)
abc   :   5.749951   7.852739   7.852739
angles: 114.245155 104.127123 104.127123
Sites (14)
  #  SP           a         b         c  Wyckoff
---  ----  --------  --------  --------  ---------
  0  Ho    0.275216  0.870381  0.251299  6f
  1  Pd    0         0         0         1a
  2  Pd    0.5       0         0         1b
  3  Pd    0.549153  0.319948  0.485194  6f

In [48]:
sym.wyckoff_symbols

['6f', '1a', '1b', '6f']

AttributeError: 'SymmetrizedStructure' object has no attribute 'get_symmetry_operations'

In [43]:
len(sym)

14

In [36]:
sym.find_equivalent_sites(sym[0])

[PeriodicSite: Ho0 (Ho) (-0.892, 5.74, -1.22) [0.2752, 0.8704, 0.2513],
 PeriodicSite: Ho1 (Ho) (1.195, 2.512, -0.7786) [0.4048, 0.3809, 0.1296],
 PeriodicSite: Ho2 (Ho) (-1.954, 4.938, 2.414) [0.02392, 0.7487, 0.6191],
 PeriodicSite: Ho3 (Ho) (3.68, 0.8548, 4.444) [0.7248, 0.1296, 0.7487],
 PeriodicSite: Ho4 (Ho) (1.593, 4.083, 4.003) [0.5952, 0.6191, 0.8704],
 PeriodicSite: Ho5 (Ho) (4.742, 1.657, 0.811) [0.9761, 0.2513, 0.3809]]

In [38]:
spa = SpacegroupAnalyzer(sym)

In [40]:
spa.get_space_group_operations()

[SymmOp(self.affine_matrix=array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])),
 SymmOp(self.affine_matrix=array([[-1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.],
        [ 0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  1.]])),
 SymmOp(self.affine_matrix=array([[ 1.,  0., -1.,  0.],
        [ 0.,  0., -1.,  0.],
        [ 0.,  1., -1.,  0.],
        [ 0.,  0.,  0.,  1.]])),
 SymmOp(self.affine_matrix=array([[-1.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]])),
 SymmOp(self.affine_matrix=array([[ 1., -1.,  0.,  0.],
        [ 0., -1.,  1.,  0.],
        [ 0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.]])),
 SymmOp(self.affine_matrix=array([[-1.,  1.,  0.,  0.],
        [ 0.,  1., -1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.]]))]

In [ ]:
s